In [5]:
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split

SEED = np.random.seed(0)
DATA = Path('data')
TARGETS = ['participants', 'interventions', 'outcomes']
SUBSET = 'Train' # 'Test'
N_TRAIN_DOCS = 4500
BASE_FILENAME = 'raw'
SAVE_LOC = Path('train/')

%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


# todo

- could try document selection. Remove titles?

# Load feature set and labels

- clusterings
- tf idf?
- top 100 hotcoded keywords from the BOW?
- sentence level information?
- Stanford?

In [6]:
FEATURES = ['base', 'pubmedFT', 'pos'] # 'rawFT', 'clust' // base not needed?
feat_paths = [f'data\\features\\{feature}.pkl' for feature in FEATURES]
labels_path = DATA / 'raw' / 'labels.pkl'

print(f"Loading feature set from {', '.join(feat_paths)}")
feats_in = pd.concat([pd.read_pickle(path) for path in feat_paths], axis=1)
      
print(f'Loading labels from {labels_path}')
labels_in = pd.read_pickle(labels_path)
      
# data_mem = sum(sys.getsizeof(i) for i in [X,y]) # slow command
# print(f'Loaded {data_mem / (10**9)} GB')

data = pd.concat([feats_in.drop('Word',axis=1), labels_in],axis=1)

assert not data.columns.duplicated().any()
      
print('Data load complete.')

Loading feature set from data\features\base.pkl, data\features\pubmedFT.pkl, data\features\pos.pkl
Loading labels from data\raw\labels.pkl
Data load complete.


# Train/Test split

Withhold part of the data for evaluation. Create a script for this.

In [3]:
%%notify

feats = [col for col in data.columns if col not in TARGETS]
feats.remove('Word')
targets = TARGETS
hold_size = 10 # 493
k_folds = 5 # only simulated for now

doc_ids = list(data.index.unique(level='doc'))

train_val_idx, hold_idx = train_test_split(doc_ids, train_size=N_TRAIN_DOCS, test_size=hold_size)

word = data
test_hold = data.loc[(hold_idx, slice(None)),:].drop('Word',axis=1) # slow; comment when testing
train_val = data.loc[(train_val_idx, slice(None)),:].drop('Word',axis=1)

NameError: name 'train_test_split' is not defined

<IPython.core.display.Javascript object>

In [4]:
train_val.to_pickle('data/split/train.pkl') # add mkdir
test_hold.to_pickle('data/split/test.pkl')

NameError: name 'train_val' is not defined

Validation split (KCV currently not implemented, so just a simulation)

In [ ]:
%%notify

def hotcode(df):
    
    print(df.columns)
    num_cols = df._get_numeric_data().columns
    
    cols = set(df.columns)
    cat_cols = (set(df.columns) - set(num_cols)) #- set('Word')
    
    print((cat_cols))
    dummies = pd.get_dummies(df[cat_cols])
    
    print('hotcode complete.')
    # assert check that type is numeric for all
    
    return pd.concat([dummies, df[num_cols]], axis=1)
    
pio = {"participants", "interventions", "outcomes"}
features = set(train_val.columns).difference(pio.union({'Word'}))

# print('Word' in features)
# print(train_val.shape)

print('hotcoding categorical columns...')
try:
    train_val = hotcode(train_val)
except ValueError:
    print('No categorical values found in data')

In [ ]:
train_val.to_parquet()

- produce some downsampled data
- can produce different feature sets

# Downsampling

In [ ]:
# to dataset preparation script

n_docs = 1000

doc_ids = data.index.unique('doc').values.tolist()
ds = random.sample(doc_ids, n_docs)
data_ds = data.loc[(ds,slice(None)),:]